# ASTR 19 Final Project

## Step 3: Following the Tutorial

Import libraries & set up

In [ ]:
import numpy as np
import sep as sep
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

Import matplotlib style guide

In [ ]:
plt.style.use("Downloads/astr19_matplotlib_defaults.txt")

### Reading an example image from FITS file and display it

Use astropy to open the FITS file and get its data.

In [ ]:
#read image into standard 2-d numpy array
data = fits.open("Downloads/image.fits")
image_data = fits.getdata("Downloads/image.fits", ext=0)

Use numpy to find the mean and standard deviation of the image data and show it on a figure with a colorbar.

In [ ]:
#show the image
m, s = np.mean(image_data), np.std(image_data)
plt.imshow(image_data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
#save the imaege as a PNG
plt.savefig('final-project-image1.png', bbox_inches='tight', dpi=400, facecolor='white')

## Background subtraction in sep

background estimation & source detection are separate steps in SEP

In [ ]:
#measure a spatially varying background on the image
bkg = sep.Background(image_data)

In [ ]:
#get a 'global' mean and noise of the image background
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
#evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()
# bkg_image = np.array(bkg) is equivalent to above

In [ ]:
#show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
#save the image as a PNG
plt.savefig('final-project-image2.png', bbox_inches='tight', dpi=400, facecolor='white')

In [ ]:
#evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()

In [ ]:
#show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
#save the image as a PNG
plt.savefig('final-project-image3.png', bbox_inches='tight', dpi=400, facecolor='white')

In [ ]:
#subtract the background
data_sub = image_data - bkg

## Object detection on background-subtracted data
### setting detection threshold to be a constant value of 1.5σ

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

In [ ]:
#how many objects were detected
len(objects)

In [ ]:
from matplotlib.patches import Ellipse

#plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

#plot an ellipse for each object
for i in range (len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                    width=6*objects['a'][i],
                    height=6*objects['b'][i],
                    angle=objects['theta'][i] * 180./np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
#save the image as a PNG
plt.savefig('final-project-image4.png', bbox_inches='tight', dpi=400, facecolor='white')

'objects' has many other fields!

In [ ]:
#available fields
objects.dtype.names

## Aperture photometry

Perform simple circular aperture photometry with a 3-pixel radius at the locations of the objects

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)
#flux, fluxerr, and flag are 1-d arrays with 1 entry/object

In [ ]:
#show the first 10 objects results
for i in range(10):
    print("object {:d}: flux={:f} +/- {:f}". format(i, flux[i], fluxerr[i]))